In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# 数据预处理操作，针对CIFAR10彩色图像数据集进行归一化等处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# 加载CIFAR10训练集，设置合适参数，CIFAR10的train参数为True表示加载训练集部分
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                         download=True, transform=transform)
# 加载CIFAR10测试集，train参数为False表示加载测试集部分
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

# 划分训练集和验证集，采用常见的80%训练集，20%验证集的划分方式
train_data, val_data = train_test_split(trainset, test_size=0.1, random_state=42)

# 创建训练集数据加载器，设置合理的参数，如batch_size、是否打乱数据以及使用多进程加载数据（num_workers设为合适值）
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64,
                                           shuffle=True, num_workers=2)
# 创建验证集数据加载器
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64,
                                         shuffle=False, num_workers=2)
# 创建测试集数据加载器
test_loader = torch.utils.data.DataLoader(testset, batch_size=64,
                                          shuffle=False, num_workers=2)

# 实例化预训练的ResNet18模型，CIFAR10是彩色图像，其输入通道数符合ResNet18默认的3通道输入要求，无需修改conv1层
model = resnet18(pretrained=True)
# 根据CIFAR10数据集的类别数量（10类）调整全连接层输出维度
model.fc = nn.Linear(model.fc.in_features, 10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 定义损失函数，依然采用交叉熵损失函数用于多分类任务
criterion = nn.CrossEntropyLoss()
# 定义优化器，使用Adam优化器，学习率可后续根据实际情况进一步优化调整
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    val_loss = running_loss / len(val_loader)
    val_acc = 100. * correct / total
    precision = precision_score(all_targets, all_preds, average='macro')
    recall = recall_score(all_targets, all_preds, average='macro')
    f1 = f1_score(all_targets, all_preds, average='macro')
    conf_matrix = confusion_matrix(all_targets, all_preds)
    return val_loss, val_acc, precision, recall, f1, conf_matrix


def test(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    test_acc = 100. * correct / total
    precision = precision_score(all_targets, all_preds, average='macro')
    recall = recall_score(all_targets, all_preds, average='macro')
    f1 = f1_score(all_targets, all_preds, average='macro')
    conf_matrix = confusion_matrix(all_targets, all_preds)
    return test_acc, precision, recall, f1, conf_matrix


epochs = 5
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, precision, recall, f1, conf_matrix = validate(model, val_loader, criterion, device)
    print(f'Epoch {epoch + 1}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, '
          f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')
    print("Confusion Matrix:")
    print(conf_matrix)

test_acc, precision, recall, f1, conf_matrix = test(model, test_loader, device)
print(f"Test Accuracy: {test_acc:.2f}%")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-score: {f1:.4f}")
print("Test Confusion Matrix:")
print(conf_matrix)

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: Train Loss: 0.9870, Train Acc: 66.91%, Val Loss: 0.8043, Val Acc: 73.00%, Precision: 0.7316, Recall: 0.7330, F1-score: 0.7268
Confusion Matrix:
[[351  16  14  16   7   2   4   3  30  33]
 [  7 427   0   2   2   0   2   1  14  32]
 [ 37   6 347  12  30  27  44  10   9   5]
 [  5  10  25 209  23 156  65   6  13  11]
 [  7   2  21  21 363  17  34  15   2   4]
 [  1   7 103  32  25 292  17  18   2   7]
 [  1   5  17   6   8  10 433   1   3   1]
 [ 14   2  17  23  35  34   5 354   6  19]
 [ 26  14   6   0   0   1   3   0 441  12]
 [  3  32   2   4   2   1  10   0  13 433]]
Epoch 2: Train Loss: 0.6798, Train Acc: 77.36%, Val Loss: 0.8123, Val Acc: 72.68%, Precision: 0.7495, Recall: 0.7279, F1-score: 0.7223
Confusion Matrix:
[[415   7  25   6   2   2   1   8   9   1]
 [ 27 406   8   2   2   2  12   4  22   2]
 [ 36   0 422  20   7   5  24  10   2   1]
 [ 22   1  45 330  19  26  36  38   5   1]
 [ 20   1  68  16 315   7  15  41   3   0]
 [  6   2  62 156  20 183  23  51   1   0]
 [  7